In [34]:
import pandas as pd
import os

In [35]:
caminho = os.path.join('..', '..', 'db_softtek') 
path_arq = os.path.abspath(caminho)

In [36]:
db_orig = '\\FIAP_DataBase_v1.xlsx'  # Substitua pelo nome do seu arquivo


In [37]:
# Carrega tabela de demandas:
tbl_demandas = pd.read_excel(path_arq+db_orig, 'DEMANDAS').drop_duplicates()

# Carrega tabela de Contratos:
tbl_contr = pd.read_excel(path_arq+db_orig, 'CONTRATOS').drop_duplicates()

# Carrega tabela de Custos:
tbl_custos = pd.read_excel(path_arq+db_orig, 'CUSTOS').drop_duplicates()
tbl_custos = tbl_custos.groupby(['EXERCICIO','PERIODO','CODIGO_AT']).agg({'VALOR_AT':'mean'}).reset_index()

# Carrega tabela de Equipe:
tbl_equipe = pd.read_excel(path_arq+db_orig, 'EQUIPE').drop_duplicates()


In [38]:
#Demandas x Equipe:

tbl_1 = pd.merge(tbl_demandas, tbl_equipe, how ='left', on = 'IS').fillna(0).drop_duplicates()

# view1 x Custos:
tbl_2  = pd.merge(tbl_1, tbl_custos, how ='left', on = ['EXERCICIO','PERIODO','CODIGO_AT']).drop_duplicates()

In [39]:
#Cruzamento por tbl_2 x contratos:

tbl_3  = pd.merge(tbl_2, tbl_contr, how ='left', left_on = ['PROJETO'],
                  right_on=['PROJETO'])


# Tirando duplicatas por contains de tipo chamado x tipo_demanda:

tbl_3['esta_contido'] = ''
for index, row in tbl_3.iterrows():
    if row['TIPO_CHAMADO'] in row['TIPO_DEMANDA']:
        tbl_3.at[index, 'esta_contido'] = 'Sim'

tbl_3 = tbl_3[tbl_3['esta_contido'] =='Sim'].reset_index().drop(columns=['index','esta_contido']).fillna(0)

tbl_3.describe()

,Column1,EXERCICIO,PERIODO,DIA,HORAS,VALOR_AT,BASELINE,VALOR CONTRATO,EXPERT,CUSTO_VENDA_EXPERT,...,PLENO,CUSTO_VENDA_PLENO,JUNIOR,CUSTO_VENDA_JUNIOR,ESTAGIARIO,CUSTO_VENDA_ESTAGIARIO,HORAS_GESTAO,CUSTO_VENDA_GESTAO,INICIO_CONTRATO,FIM_CONTRATO
count,999.000000,999.0,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,9.990000e+02,...,999.000000,9.990000e+02,999.000000,999.00,999.000000,9.990000e+02,999.000000,999.00,999,999
mean,10.310310,2024.0,1.972973,15.298298,2.383504,43.691570,217.889890,14147.427427,0.038238,7.178000e+01,...,0.339740,5.313000e+01,0.198498,24.15,0.160260,1.449000e+01,32.683483,77.02,2024-01-01 00:00:00,2024-12-31 00:00:00
min,1.000000,2024.0,1.000000,1.000000,0.140000,13.636364,70.000000,1680.000000,0.000000,7.178000e+01,...,0.000000,5.313000e+01,0.100000,24.15,0.000000,1.449000e+01,10.500000,77.02,2024-01-01 00:00:00,2024-12-31 00:00:00
25%,2.000000,2024.0,1.000000,8.000000,0.700000,22.727273,252.000000,16800.000000,0.000000,7.178000e+01,...,0.400000,5.313000e+01,0.100000,24.15,0.000000,1.449000e+01,37.800000,77.02,2024-01-01 00:00:00,2024-12-31 00:00:00
50%,7.000000,2024.0,2.000000,15.000000,1.400000,36.363636,252.000000,16800.000000,0.000000,7.178000e+01,...,0.400000,5.313000e+01,0.100000,24.15,0.100000,1.449000e+01,37.800000,77.02,2024-01-01 00:00:00,2024-12-31 00:00:00
75%,14.000000,2024.0,3.000000,22.000000,2.800000,63.636364,280.000000,19600.000000,0.100000,7.178000e+01,...,0.500000,5.313000e+01,0.100000,24.15,0.100000,1.449000e+01,42.000000,77.02,2024-01-01 00:00:00,2024-12-31 00:00:00
max,49.000000,2024.0,3.000000,31.000000,13.580000,100.000000,280.000000,19600.000000,0.100000,7.178000e+01,...,0.500000,5.313000e+01,0.500000,24.15,0.500000,1.449000e+01,42.000000,77.02,2024-01-01 00:00:00,2024-12-31 00:00:00
std,10.712391,0.0,0.828035,8.353123,2.619132,24.861009,85.441538,7232.533191,0.048621,2.843595e-14,...,0.199077,7.108986e-15,0.172416,0.00,0.199077,1.777247e-15,12.816231,0.00,NaN,NaN


In [43]:
tbl_3.head()

,PROJETO,CHAMADO,Column1,DT_ABERTURA_CHAMADO,MODULO_CHAMADO,TIPO_CHAMADO,STATUS_CHAMADO,IS,CONSULTOR,MODULO_CustosNSULTOR,...,PLENO,CUSTO_VENDA_PLENO,JUNIOR,CUSTO_VENDA_JUNIOR,ESTAGIARIO,CUSTO_VENDA_ESTAGIARIO,HORAS_GESTAO,CUSTO_VENDA_GESTAO,INICIO_CONTRATO,FIM_CONTRATO
0,PROJETO_01,TASK105721218-1,6,2024-03-25 11:29:08,Financeiro,Melhoria,UAT - M,ARDA2,Ali,Vendas,...,0.4,53.13,0.1,24.15,0.1,14.49,37.8,77.02,2024-01-01,2024-12-31
1,PROJETO_01,TASK105614218-1,3,2023-12-11 08:59:42,Financeiro,Suporte,Closed,ASGN,Ari,Financeiro,...,0.4,53.13,0.1,24.15,0.1,14.49,37.8,77.02,2024-01-01,2024-12-31
2,PROJETO_01,TASK105614218-1,3,2023-12-11 08:59:42,Financeiro,Suporte,Closed,ASGN,Ari,Financeiro,...,0.4,53.13,0.1,24.15,0.1,14.49,37.8,77.02,2024-01-01,2024-12-31
3,PROJETO_01,TASK105614218-1,3,2023-12-11 08:59:42,Financeiro,Suporte,Closed,ASGN,Ari,Financeiro,...,0.4,53.13,0.1,24.15,0.1,14.49,37.8,77.02,2024-01-01,2024-12-31
4,PROJETO_01,TASK105700511-1,5,2024-03-08 14:15:06,Financeiro,Suporte,Closed,ASGN,Ari,Financeiro,...,0.4,53.13,0.1,24.15,0.1,14.49,37.8,77.02,2024-01-01,2024-12-31


In [41]:
#Sainda primeira Tabela, analitica Full:

tbl_3.to_excel(path_arq+'\\cubo_anl\\tbl_full_analitica.xlsx',index=False)